In [1]:
import time
import random
import numpy as np
import pandas as pd
import torch
import os

from custom_trading_env import TradingEnv
import DQNTradingAgent.dqn_agent as dqn_agent
from arguments import argparser

In [2]:
df = pd.read_hdf('./dataset/binance_data_test.h5', 'STW')
df.fillna(method='ffill', inplace=True)

sample_len   = len(df)
obs_data_len = 192
step_len     = 1
n_action_intervals = 5
init_budget = 1
fee = 0.005

env = TradingEnv(custom_args=object(), env_id='custom_trading_env', obs_data_len=obs_data_len, step_len=step_len, sample_len=sample_len,
                 df=df, fee=fee, initial_budget=1, n_action_intervals=n_action_intervals, deal_col_name='c', sell_at_end=True,
                 feature_names=['o', 'h','l','c','v',
                                'num_trades', 'taker_base_vol'])

[2019-07-20 10:15:04,515] Making new env: custom_trading_env


In [3]:
total_volume = []
total_fee = []
for i in range(1,27+1):
    save_location = 'saves/transfer0/0.003/{}'.format(i)

    device = torch.device("cpu")
    dqn_agent.set_device(device)
    load_weight_n = 500

    state = env.reset()
    # env.render()

    hyperparams = torch.load(os.path.join(save_location, "hyperparams.pth"))

    agent = dqn_agent.Agent(action_size=2 * n_action_intervals + 1, obs_len=obs_data_len, num_features=env.reset().shape[-1], **hyperparams)
    agent.qnetwork_local.load_state_dict(torch.load(os.path.join(save_location, 'TradingGym_Rainbow_{:d}.pth'.format(load_weight_n)), map_location=device))
    agent.qnetwork_local.to(device)
    agent.qnetwork_local.eval()

    done = False

    while not done:

        action = int(agent.act(state))
        state, reward, done, info = env.step(action)
    #     print()
    #     print("Action:", action)
    #     print("Budget:", env.budget)
    #     print("Reward:", reward)
        if done:
            break

    total_volume.append(info['position_variation'].abs().sum())
    info.to_csv(f"Results/{str(fee)}/info_{i}.csv")
    total_fee.append(env.total_fee)

/home/jeffrey/Binanace_trading_simulation/TradingGym/custom_trading_env.py:100: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.price = self.df_sample[self.price_name].as_matrix()
/home/jeffrey/Binanace_trading_simulation/TradingGym/custom_trading_env.py:102: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.obs_features = self.df_sample[self.using_feature].as_matrix()


FileNotFoundError: [Errno 2] No such file or directory: 'saves/transfer0/0.003/10/hyperparams.pth'

In [ ]:
len(total_fee)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(total_fee)
plt.grid()
plt.title("Total fee")
plt.xlabel("N_agent")

In [ ]:
plt.plot(total_volume)
plt.grid()
plt.title("Total Volume")
plt.xlabel("N_agent")

In [ ]:
import pickle as pkl

In [ ]:
pkl.

In [8]:
for i in range(1,27+1):
    if not os.listdir(f'saves/transfer0/0.003/{i}/'):
        print(i)

10
12


In [5]:
os.path.exists('saves/transfer0/0.003/10/')

True

## 총 거래량

In [5]:
info['position_variation'].abs().sum()

1125.6077348048857

In [6]:
info.to_csv(f"Results/{1}.csv")

## 총 수수료

In [7]:
env.total_fee

0.010933610079228023